# Command Pipeline

<a target="_blank" href="https://colab.research.google.com/github/superwise-ai/dsl-spa/blob/main/examples/command_pipeline_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

A Command Pipeline is used to define set of commands that are executed based on an LLM response. The LLM response should detail what command to call (typically with the field `base.command_name`). Each command is defined as a set of Actions. Each Action is defined with a mapping to a python function and the set of arguments to input to that python function.

In this example, we will create a pipeline that takes in a command, in this case either `get_time`, which gets the current time in a timezone, or `get_fahrenheit` and `get_celsius` which converts temperature from celsius to fahrenheit or fahrenheit to celsius.

# Install Dependencies

In [ ]:
%pip install dsl-spa==0.4.1

# Create Pipeline Schema

### Import Schema Definitions

In [1]:
from dsl_spa.utils.schema import PipelineField,Action,Command,CommandPipelineSchema

### Create Fields

Define `base.command_name` as required because the pipeline should always be passed a command to execute. `base.timezone`, `base.temperature_value`, and `base.temperature_scale` are not required as the pipeline should run if `base.timezone`, `base.temperature_value`, or `base.temperature_scale` is not included.

In [2]:
command_name = PipelineField(field_name="base.command_name", 
                             field_type="string",
                             required=True,
                             description="Name of the command to execute")
timezone = PipelineField(field_name="base.timezone", 
                         field_type="string",
                         required=False,
                         description="Timezone to get time of")
temperature = PipelineField(field_name="base.temperature_value", 
                            field_type="float",
                            required=False,
                            description="temperature value to convert")
scale = PipelineField(field_name="base.temperature_scale", 
                      field_type="str",
                      required=False,
                      description="temperature scale to convert temperature_value to")
fields = [command_name,timezone,temperature,scale]

For this pipeline, we defined the field holding the command to run as `command_name`. This is the default name for the field in DSL-SPA. However, one can name this field whatever they want (helpful for adjusting the field names based on the context your LLM application uses). Just need to make sure the new field containing the name of the command is passed to the pipeline when calling `process_command`.

### Create Actions

It's important that when defining an Action, the `function_name` maps to a key in the `function_dict` defined later.

In [3]:
get_time_action = Action(name="get_time_in_timezone",function_name="get_time", output_field="output")
get_time_action.add_attribute("timezone", "base.timezone", optional=False)

get_temperature_action = Action(name="convert_temperature",function_name="convert_temp", output_field="output")
get_temperature_action.add_attribute("temperature", "base.temperature_value", optional=False)
get_temperature_action.add_attribute("scale", "base.temperature_scale", optional=False)

actions = [get_time_action, get_temperature_action]

### Create Commands

In [4]:
get_time_command = Command(name = "get_time",actions=[get_time_action])
get_temperature_command = Command(name = "get_temp",actions=[get_temperature_action])
commands = [get_time_command,get_temperature_command]

### Create Command Pipeline Schema

In [5]:
pipeline_schema = CommandPipelineSchema(pipeline_name="timezone_pipeline",
                                        fields=fields,
                                        actions=actions,
                                        commands=commands)
schema = pipeline_schema.get_schema()
schema

{'pipeline_name': 'timezone_pipeline',
 'fields': {'base': {'base.command_name': {'name': 'base.command_name',
    'type': 'string',
    'required': True,
    'description': 'Name of the command to execute'},
   'base.timezone': {'name': 'base.timezone',
    'type': 'string',
    'required': False,
    'description': 'Timezone to get time of'},
   'base.temperature_value': {'name': 'base.temperature_value',
    'type': 'float',
    'required': False,
    'description': 'temperature value to convert'},
   'base.temperature_scale': {'name': 'base.temperature_scale',
    'type': 'str',
    'required': False,
    'description': 'temperature scale to convert temperature_value to'}}},
 'actions': [{'name': 'get_time_in_timezone',
   'function': 'get_time',
   'attributes': [{'name': 'timezone',
     'field': 'base.timezone',
     'optional': False}],
   'output_field': 'output'},
  {'name': 'convert_temperature',
   'function': 'convert_temp',
   'attributes': [{'name': 'temperature',
     '

# Create Pipeline

### Define custom function

In [6]:
from datetime import datetime
from zoneinfo import ZoneInfo

In [7]:
def get_time_in_timezone(timezone: str):
    if timezone.lower() == "utc":
        time = datetime.now(tz=ZoneInfo("UTC"))
        return f"The time is {time}"
    elif timezone.lower() in ["et","est","edt","eastern"]:
        time = datetime.now(tz=ZoneInfo("US/Eastern"))
        return f"The time is {time}"
    elif timezone.lower() in ["ct","cst","cdt","central"]:
        time = datetime.now(tz=ZoneInfo("US/Eastern"))
        return f"The time is {time}"
    elif timezone.lower() in ["mt","mst","mdt","mountain"]:
        time = datetime.now(tz=ZoneInfo("US/Mountain"))
        return f"The time is {time}"
    elif timezone.lower() in ["pt","pst","pdt","pacific"]:
        time = datetime.now(tz=ZoneInfo("US/Pacific"))
        return f"The time is {time}"
    else:
        try:
            return datetime.now(tz=ZoneInfo(timezone))
        except:
            return "Could not determine timezone"

In [8]:
def convert_temperature(temperature: float, scale: str):
    if scale.lower() in ["f","fahrenheit"]:
        return f"The temperature is: {temperature*1.8 + 32} degrees Fahrenheit"
    elif scale.lower() in ["c", "celsius"]:
        return f"The temperature is: {(temperature-32)/1.8} degrees Celsius"
    else:
        return "Temperature Scale Not Recognized"

### Test functions

In [9]:
print(get_time_in_timezone("eastern"))
print(get_time_in_timezone("utc"))
print(get_time_in_timezone("northern"))

The time is 2025-12-17 14:17:53.843711-05:00
The time is 2025-12-17 19:17:53.844713+00:00
Could not determine timezone


In [10]:
print(convert_temperature(100,"celsius"))
print(convert_temperature(10,"f"))
print(convert_temperature(200,"kelvin"))

The temperature is: 37.77777777777778 degrees Celsius
The temperature is: 50.0 degrees Fahrenheit
Temperature Scale Not Recognized


### Create Custom Function Dictionary

Make sure the keys in the function_dict map to the function_name values given to the Actions earlier.

In [11]:
functions_dict = {
    "get_time": get_time_in_timezone,
    "convert_temp": convert_temperature
}

### Create Mock LLM Outputs (Fields)
These mock outputs will act as example outputs from our LLM.

In [12]:
fields_time_1 = {
    "command_name": "get_time",
    "timezone": "MT"
}
fields_time_2 = {
    "command_name": "get_time",
    "timezone": "Europe/Isle_of_Man"
}
fields_time_3 = {
    "command_name": "get_time",
    "timezone": "mars"
}


fields_temp_1 = {
    "command_name": "get_temp",
    "temperature_value": 78,
    "temperature_scale": "celsius"
}
fields_temp_2 = {
    "command_name": "get_temp",
    "temperature_value": -12,
    "temperature_scale": "f"
}
fields_temp_3 = {
    "command_name": "get_temp",
    "temperature_value": 78
}

### Test Pipeline

In [13]:
from dsl_spa.pipeline.pipeline import CommandPipeline, PipelineException


Let's test the pipeline for getting the time.

Because the field holding the command name is `base.command_name`, `process_command` can be called without any arguments. If though, our command was stored in a field called `base.user_request`, `process_command` would need to be called as:
```
pipeline.process_command("base.user_request")
```

In [14]:
pipeline = CommandPipeline(fields_input_dict=fields_time_1, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
pipeline.process_command()
pipeline.get_command_result(output_field="output")

'The time is 2025-12-17 12:17:54.598951-07:00'

In [15]:
pipeline = CommandPipeline(fields_input_dict=fields_time_2, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
pipeline.process_command()
pipeline.get_command_result(output_field="output")

datetime.datetime(2025, 12, 17, 19, 17, 54, 608384, tzinfo=zoneinfo.ZoneInfo(key='Europe/Isle_of_Man'))

In [16]:
pipeline = CommandPipeline(fields_input_dict=fields_time_3, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
pipeline.process_command()
pipeline.get_command_result(output_field="output")

'Could not determine timezone'

Now let's test the pipeline for converting temperature.

In [17]:
pipeline = CommandPipeline(fields_input_dict=fields_temp_1, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
pipeline.process_command()
pipeline.get_command_result(output_field="output")

'The temperature is: 25.555555555555554 degrees Celsius'

In [18]:
pipeline = CommandPipeline(fields_input_dict=fields_temp_2, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
pipeline.process_command()
pipeline.get_command_result(output_field="output")

'The temperature is: 10.399999999999999 degrees Fahrenheit'

In [19]:
pipeline = CommandPipeline(fields_input_dict=fields_temp_3, json_schema=schema, connectors={}, functions_dict=functions_dict)
pipeline.initiate_command_pipeline()
try:
    pipeline.process_command()
    pipeline.get_command_result(output_field="output")
except PipelineException as e:
    print(e)

Pipeline Exception: Missing fields: base.temperature_scale
